In [ ]:
import os
import pathlib
import re
import sys

import findspark
from operator import add
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, Row, SparkSession
from pyspark.sql.functions import regexp_replace, col, trim, lower, desc

prj_home = pathlib.Path().resolve().parent.parent
spark_home = os.path.join(prj_home / 'spark-3.5.0-bin-hadoop3')
findspark.init(spark_home)

sc = SparkContext(master='', appName='PySparkSqlContext') # machine that spark runs and the number of worker threads
ssc = StreamingContext(sparkContext=sc, batchDuration=20)

In [ ]:
def get_spark_session_interface(sparkConf):
    if 'sparkSessionSingletonInstance' not in globals():
        globals()['sparkSessionSingletonInstance'] = SparkSession.builder.config(conf=sparkConf).getOrCreate()
    return globals()['sparkSessionSingletonInstance']

In [ ]:
host = '127.0.0.1'
port = '5555'

lines = ssc.socketTextStream(host, int(port))
seprated = lines.flatMap(lambda l: l.split(' '))


In [ ]:
def process(time, rdd):
    print(f"-.-.-.-.-.-.-.-.-.- %s -.-.-.-..-.-.-..-.-.-" % str(time))
    if not rdd.isEmpty():
        session = get_spark_session_interface(rdd.context.getConf())
        rows_data = rdd.map(lambda word: Row(word_col=word))
        words_df = session.createDataFrame(rows_data)
        words_df.createOrReplaceTempView('words')

        word_count_df = session.sql('select word_col, count(*) from words group by word_col')
        word_count_df.show()
    
    else:
        print("No stream received!")

seprated.foreachRDD(process)

In [ ]:
ssc.start()

In [ ]:
# ssc.stop(stopGraceFully=True, stopSparkContext=True)
ssc.awaitTerminationOrTimeout(timeout=100)